In [1]:
import pandas as pd
import numpy as np

X = pd.read_csv('../../data/ZooScanSet/taxa.csv', usecols=['objid', 'taxon'], dtype={'objid': int, 'taxon': str})

In [2]:
# print( X.taxon.value_counts().index)
# print( X.taxon.value_counts().values)
#print( X.taxon.value_counts())


In [1]:
import cv2
import numpy as np

X['width'] = 0
X['height'] = 0

for index in X.index:
    filename = X['objid']
    taxon = X['taxon']
    path ='../../data/ZooScanSet/imgs/'+taxon+'/'+str(filename)+'.jpg'
    img = cv2.imread(path)
    imgdim = np.shape(img)        
    X.at[[index], 'height'] = imgdim[0]
    X.at[[index], 'width'] = imgdim[1]
    img.close()
    

NameError: name 'X' is not defined

In [1]:
for index in taxon.index:
    value = taxon[index]
    path ='../../data/ZooScanSet/imgs/Rhopalonema/'+str(value)+'.jpg'
    img = cv2.imread(path)
    imgdim = np.shape(img)        
    X.at[[index], 'height'] = imgdim[0]
    X.at[[index], 'width'] = imgdim[1]        


NameError: name 'X' is not defined

In [ ]:
%matplotlib inline
print('Mean height of Rhopalonema class: ' + str(rhopa['height'].mean()))
print('STD of height of Rhopalonema class: ' + str(rhopa['height'].std()))
print('Maximum height of Rhopalonema class: ' + str(rhopa['height'].max()))
print('Minimum height of Rhopalonema class: ' + str(rhopa['height'].min()))

print('Mean width of Rhopalonema class: ' + str(rhopa['width'].mean()))
print('STD of width of Rhopalonema class: ' + str(rhopa['width'].std()))
print('Maximum width of Rhopalonema class: ' + str(rhopa['width'].max()))
print('Minimum width of Rhopalonema class: ' + str(rhopa['width'].min()))

rhopa.plot(x='width', y='height', style='o')

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split

y = pd.get_dummies(X['taxon'])

print(np.shape(X['objid'].values))
print(np.shape(y.values))

X_train, X_test, y_train, y_test = train_test_split(X['objid'].values, y.values,
                                                   test_size=.1, random_state=42,
                                                   stratify=y)

print(np.shape(X_train))

(1433278,)
(1433278, 93)
(1289950,)


In [7]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras import Sequential
import numpy as np
from datagen import DataGenerator

# import ResNet50 with imagenet weights, skip the top layer
# as we'll make our own shortly
base_model = ResNet50(weights='imagenet', include_top=False)

# add our own stuff
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(93, activation='softmax')(x)

# model for training
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
    
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')


epochs = 10
bs = 64

# Parameters
params = {'dim': (32,32,32),
          'batch_size': 64,
          'n_classes': 6,
          'n_channels': 1,
          'shuffle': True}

training_generator = DataGenerator(X_train, y_train, **params)
validation_generator = DataGenerator(X_test, y_test, **params)


model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=4)


Epoch 1/1


TypeError: 'tuple' object cannot be interpreted as an index

In [ ]:

img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]
